In [0]:
%run "./config"

In [0]:
spark.conf.set(
    "fs.azure.account.key.storekm01.dfs.core.windows.net",
    f'{key}'
)

In [0]:
display(dbutils.fs.ls("abfss://rawdata@storekm01.dfs.core.windows.net/"))

path,name,size,modificationTime
abfss://rawdata@storekm01.dfs.core.windows.net/2025-01-01/,2025-01-01/,0,1736093953000
abfss://rawdata@storekm01.dfs.core.windows.net/2025-01-02/,2025-01-02/,0,1736093973000
abfss://rawdata@storekm01.dfs.core.windows.net/2025-01-03/,2025-01-03/,0,1736094000000


In [0]:
dbutils.widgets.text('Date','')
input_value= dbutils.widgets.get('Date')

In [0]:
import pyspark
# Create a spark session using getOrCreate() function 
from pyspark.sql import SparkSession 
spark_session = SparkSession.builder.getOrCreate()

In [0]:
df_sales = spark_session.read.format( 
    "csv").option( 
    "header", True).load(f'{raw_folder_path}/salesorder1.csv') 

In [0]:
df_sales.show()

+------------+------------------+--------+---------+---------+-----------------+---------+--------------------+--------------------+
|SalesOrderID|SalesOrderDetailID|OrderQty|ProductID|UnitPrice|UnitPriceDiscount|LineTotal|             rowguid|        ModifiedDate|
+------------+------------------+--------+---------+---------+-----------------+---------+--------------------+--------------------+
|       71774|            110562|       1|      836|  356.898|                0|  356.898|e3a1994c-7a68-4ce...|2008-06-01T00:00:...|
|       71774|            110563|       1|      822|  356.898|                0|  356.898|5c77f557-fdb6-43b...|2008-06-01T00:00:...|
|       71776|            110567|       1|      907|     63.9|                0|     63.9|6dbfe398-d15d-425...|2008-06-01T00:00:...|
|       71780|            110616|       4|      905|  218.454|                0|  873.816|377246c9-4483-48e...|2008-06-01T00:00:...|
|       71780|            110617|       2|      983|  461.694|       

In [0]:
from pyspark.sql.functions import col,concat_ws, current_timestamp, lit, split,to_date
df_sales_modified= df_sales.withColumn('ModifiedDate', to_date(col('ModifiedDate')))\
               .drop('rowguid')\
               .withColumn("Date",lit(input_value))\
               .withColumnRenamed("ModifiedDate","Salesdate")


In [0]:
df_sales_modified.show()

+------------+------------------+--------+---------+---------+-----------------+---------+----------+----+
|SalesOrderID|SalesOrderDetailID|OrderQty|ProductID|UnitPrice|UnitPriceDiscount|LineTotal| Salesdate|Date|
+------------+------------------+--------+---------+---------+-----------------+---------+----------+----+
|       71774|            110562|       1|      836|  356.898|                0|  356.898|2008-06-01|    |
|       71774|            110563|       1|      822|  356.898|                0|  356.898|2008-06-01|    |
|       71776|            110567|       1|      907|     63.9|                0|     63.9|2008-06-01|    |
|       71780|            110616|       4|      905|  218.454|                0|  873.816|2008-06-01|    |
|       71780|            110617|       2|      983|  461.694|                0|  923.388|2008-06-01|    |
|       71780|            110618|       6|      988|  112.998|              0.4| 406.7928|2008-06-01|    |
|       71780|            110619|    

In [0]:
df_sales_modified.printSchema()

root
 |-- SalesOrderID: string (nullable = true)
 |-- SalesOrderDetailID: string (nullable = true)
 |-- OrderQty: string (nullable = true)
 |-- ProductID: string (nullable = true)
 |-- UnitPrice: string (nullable = true)
 |-- UnitPriceDiscount: string (nullable = true)
 |-- LineTotal: string (nullable = true)
 |-- Salesdate: date (nullable = true)
 |-- Date: string (nullable = false)



In [0]:
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType, StringType, DoubleType,DateType

df_sales_data = (df_sales_modified
                   .withColumn("SalesOrderID", col("SalesOrderID").cast(IntegerType()))
                   .withColumn("SalesOrderDetailID", col("SalesOrderDetailID").cast(IntegerType()))
                   .withColumn("OrderQty", col("OrderQty").cast(IntegerType()))
                   .withColumn("ProductID", col("ProductID").cast(IntegerType()))
                   .withColumn("UnitPrice", col("UnitPrice").cast(DoubleType()))
                   .withColumn("UnitPriceDiscount", col("UnitPriceDiscount").cast(DoubleType()))
                   .withColumn("LineTotal", col("LineTotal").cast(DoubleType()))
                   .withColumn("Salesdate", col("Salesdate").cast(DateType()))
                   .withColumn("Date", col("Date").cast(DateType())))

In [0]:
df_sales_data.printSchema()

root
 |-- SalesOrderID: integer (nullable = true)
 |-- SalesOrderDetailID: integer (nullable = true)
 |-- OrderQty: integer (nullable = true)
 |-- ProductID: integer (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- UnitPriceDiscount: double (nullable = true)
 |-- LineTotal: double (nullable = true)
 |-- Salesdate: date (nullable = true)
 |-- Date: date (nullable = true)



In [0]:
sales_data_processed= df_sales_data.write.mode( "overwrite").option("header", "true").csv(f'{processed_folder_path}/salesdata')